<a href="https://colab.research.google.com/github/adtitovich/dsml-17/blob/main/m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задания:
Решить задачу классификации при помощи обучения модели логистической регрессии. Качество модели оценивается путем подсчета метрик TPR, FPR и построения графиков ROC-кривой, Precision-Recall. Данные для обучения модели хранятся в файле athletes.csv, который можно найти в материалах к занятию.

#Этапы работы:
**1. Преобразуйте данные:    
a. проверьте наличие пропущенных значений. Преобразуйте/удалите пропуски по необходимости;    
b. закодируйте категориальные переменные числовыми значениями по необходимости.**

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/adtitovich/dsml-17/main/datasets/athletes.csv')
data.head()
     

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11538 non-null  int64  
 1   name         11538 non-null  object 
 2   nationality  11538 non-null  object 
 3   sex          11538 non-null  object 
 4   dob          11538 non-null  int64  
 5   height       11208 non-null  float64
 6   weight       10879 non-null  float64
 7   sport        11538 non-null  object 
 8   gold         11538 non-null  int64  
 9   silver       11538 non-null  int64  
 10  bronze       11538 non-null  int64  
 11  Dob          11538 non-null  int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 1.1+ MB


'1.5.3'

In [19]:
import datetime as dt
data['dob'] = pd.to_datetime(data['dob'], format = '%m/%d/%y')
data["dob"] = data['dob'].apply(lambda x: x - dt.timedelta(days=365*100) if x > dt.datetime.now() else x)
data['dob']=data['dob'].map(dt.datetime.toordinal)

In [20]:
data.head(20)

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,Dob
0,736041664,A Jesus Garcia,ESP,male,719112,1.72,64.0,athletics,0,0,0,719112
1,532037425,A Lam Shin,KOR,female,725272,1.68,56.0,fencing,0,0,0,725272
2,435962603,Aaron Brown,CAN,male,727345,1.98,79.0,athletics,0,0,1,727345
3,521041435,Aaron Cook,MDA,male,726834,1.83,80.0,taekwondo,0,0,0,726834
4,33922579,Aaron Gate,NZL,male,726797,1.81,71.0,cycling,0,0,0,726797
5,173071782,Aaron Royle,AUS,male,726493,1.80,67.0,triathlon,0,0,0,726493
6,266237702,Aaron Russell,USA,male,727718,2.05,98.0,volleyball,0,0,1,727718
7,382571888,Aaron Younger,AUS,male,727100,1.93,100.0,aquatics,0,0,0,727100
8,87689776,Aauri Lorena Bokesa,ESP,female,726085,1.80,62.0,athletics,0,0,0,726085
9,997877719,Ababel Yeshaneh,ETH,female,727035,1.65,54.0,athletics,0,0,0,727035


**2. Разделите выборку на обучающее (80% данных) и тестовое (20% данных) подмножества.**

**3. Постройте ROC-кривую с помощью функции sklearn.metrics. roc_curve.**

**4. Вычислите значение ROC-AUC метрики с помощью функции sklearn.metrics.roc_auc_score.**

**5. Реализуйте подсчет метрик TPR, FPR «вручную», без использования готовых функций из библиотеки sklearn.**

**6. Постройте ROC-кривую с помощью вычисленных в п. 5 метрик: объедините графики из п. 3 и п. 6 в один. Сравните, сделайте вывод.**

**7. Постройте график Precision-Recall, используя метрики, посчитанные в п. 5.**

**8. Вычислите значение ROC-AUC метрики, используя метрики, посчитанные в п. 5.**

**9. Сформулируйте выводы по проделанной работе:
a. как по полученным графикам сделать вывод о качестве модели? Как вы оцениваете обученную модель исходя из подсчитанных метрик?
b. *может ли ROC-кривая проходить ниже диагонали?**